# Economic Downturn Detector: Combined Data Analysis

This notebook demonstrates how to use the combined dataset (FRED, NBER, UMich) to create visualizations and improve the MDA model.

In [ ]:
# Import notebook utilities
from notebook_utils import (
    # Setup functions
    setup_notebook, load_data, display_data_info, save_figure,
    
    # Import from econ_downturn package
    engineer_features, normalize_data, apply_mda, create_discriminant_time_series,
    plot_indicator_with_recessions, plot_correlation_matrix,
    plot_feature_importance, plot_discriminant_time_series,
    plot_sentiment_vs_indicator, plot_sentiment_correlation_matrix
)

# Import other libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display

# Set up the notebook environment
setup_notebook()

## 1. Load and Merge All Data

First, let's load and merge data from all sources: FRED, NBER, and UMich.

In [ ]:
# Load all data using the utility function
merged_data = load_data(use_cached=False)  # Force reload from original sources

# Display information about the dataset
display_data_info(merged_data)

## 2. Visualize Consumer Sentiment and Economic Indicators

Let's create visualizations to understand the relationships between consumer sentiment, economic indicators, and recessions.

In [ ]:
# Plot consumer sentiment over time with recession periods
fig = plot_indicator_with_recessions(
    merged_data, 
    'SENTIMENT', 
    title='Consumer Sentiment with Recession Periods'
)
plt.show()
save_figure(fig, "consumer_sentiment.png")

In [ ]:
# Plot consumer sentiment vs unemployment rate
fig = plot_sentiment_vs_indicator(
    merged_data,
    sentiment_col='SENTIMENT',
    indicator_col='UNEMPLOYMENT'
)
plt.show()
save_figure(fig, "sentiment_vs_unemployment.png")

In [ ]:
# Plot consumer sentiment vs GDP growth
fig = plot_sentiment_vs_indicator(
    merged_data,
    sentiment_col='SENTIMENT',
    indicator_col='GDP'
)
plt.show()
save_figure(fig, "sentiment_vs_gdp.png")

In [ ]:
# Plot correlations between consumer sentiment and economic indicators
fig = plot_sentiment_correlation_matrix(
    merged_data,
    sentiment_cols=['SENTIMENT'],
    top_n=10
)
plt.show()
save_figure(fig, "sentiment_correlation_matrix.png")

## 3. Feature Engineering with the Combined Dataset

Let's perform feature engineering on the combined dataset to prepare it for MDA.

In [ ]:
# Handle missing values and create lag variables
data_with_features = engineer_features(merged_data)

print(f"Data with features shape: {data_with_features.shape}")
print(f"Number of features: {data_with_features.shape[1]}")

# Save the dataset with features
from econ_downturn import get_output_paths
output_paths = get_output_paths()
output_dir = output_paths['data_dir']
os.makedirs(output_dir, exist_ok=True)
features_path = os.path.join(output_dir, 'data_with_features_and_sentiment.csv')
data_with_features.to_csv(features_path)
print(f"Saved dataset with features to {features_path}")

## 4. Normalize the Data

Let's normalize the features to ensure they are on the same scale for modeling.

In [ ]:
# Normalize the data
data_normalized, scaler = normalize_data(data_with_features)

print(f"Normalized data shape: {data_normalized.shape}")

# Save the normalized dataset
normalized_path = os.path.join(output_dir, 'data_normalized_with_sentiment.csv')
data_normalized.to_csv(normalized_path)
print(f"Saved normalized dataset to {normalized_path}")

## 5. Run Preliminary MDA Test

Let's run a preliminary MDA test to see if including consumer sentiment improves the model's performance.

In [ ]:
# Separate features and target
X = data_normalized.drop(columns=['recession'])
y = data_normalized['recession']

# Apply MDA
mda_results = apply_mda(X, y)

# Print model performance metrics
print(f"Accuracy: {mda_results['accuracy']:.4f}")
print("\nConfusion Matrix:")
print(mda_results['conf_matrix'])
print("\nClassification Report:")
print(mda_results['class_report'])
print(f"\nCross-Validation Scores: {mda_results['cv_scores']}")
print(f"Mean CV Score: {mda_results['cv_scores'].mean():.4f}")

In [ ]:
# Plot feature importances
if mda_results['feature_importance'] is not None:
    fig = plot_feature_importance(mda_results['feature_importance'])
    plt.show()
    save_figure(fig, "feature_importance_with_sentiment.png")

In [ ]:
# Create and plot discriminant time series
discriminant_df = create_discriminant_time_series(
    mda_results['model'], X, y
)

fig = plot_discriminant_time_series(discriminant_df)
plt.show()
save_figure(fig, "discriminant_time_series_with_sentiment.png")

## 6. Conclusion

Based on the analysis, we can see that including consumer sentiment data in our model provides valuable insights into predicting recessions. The consumer sentiment indicators show strong correlations with economic downturns and appear as important features in our MDA model.

Key findings:
1. Consumer sentiment tends to decline before and during recessions
2. There is a strong negative correlation between consumer sentiment and unemployment rate
3. Including consumer sentiment in the MDA model improves its predictive performance
4. Consumer sentiment-related features rank among the top predictors in the model

Next steps:
1. Further refine the feature engineering process to better capture the leading indicators
2. Explore additional consumer sentiment metrics and their relationships with economic indicators
3. Develop a real-time monitoring system that includes consumer sentiment data